Detailed Results and Inferences on SBERT Architectures Using the USEEIO Dataset

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sentence_transformers import SentenceTransformer
from sklearn.metrics import accuracy_score, f1_score
import time
import torch

# Load your dataset (assuming the dataset is in CSV format)
data = pd.read_csv('useeio_dataset.csv')

# Preprocess the data (example, adjust according to your dataset)
# Assume 'text' is the column with textual descriptions and 'label' is the target variable.
features = data['text_description'].tolist()
labels = data['emission_label'].values

# Split the data into train and test sets
train_texts, test_texts, train_labels, test_labels = train_test_split(features, labels, test_size=0.2, random_state=42)


Define model

In [ ]:
def evaluate_model(model_name, train_texts, train_labels, test_texts, test_labels):
    # Load the SBERT model
    model = SentenceTransformer(model_name)

    # Encode the texts
    train_embeddings = model.encode(train_texts, convert_to_tensor=True)
    test_embeddings = model.encode(test_texts, convert_to_tensor=True)

    # Convert labels to tensor
    train_labels = torch.tensor(train_labels)
    test_labels = torch.tensor(test_labels)

    # Use a simple classifier (e.g., logistic regression)
    from sklearn.linear_model import LogisticRegression
    clf = LogisticRegression(max_iter=1000)

    # Train the classifier
    clf.fit(train_embeddings.cpu().numpy(), train_labels.cpu().numpy())

    # Predict on test set
    start_time = time.time()
    test_predictions = clf.predict(test_embeddings.cpu().numpy())
    inference_time = (time.time() - start_time) / len(test_texts)

    # Evaluate the classifier
    accuracy = accuracy_score(test_labels.cpu().numpy(), test_predictions)
    f1 = f1_score(test_labels.cpu().numpy(), test_predictions, average='weighted')

    # Print results
    print(f"Model: {model_name}")
    print(f"Accuracy: {accuracy * 100:.2f}%")
    print(f"F1 Score: {f1:.2f}")
    print(f"Inference Time: {inference_time * 1000:.2f} ms per sample")
    print("\n")

    return accuracy, f1, inference_time


In [ ]:
models = [
    'distilbert-base-nli-mean-tokens',
    'bert-base-nli-mean-tokens',
    'roberta-base-nli-mean-tokens',
    'xlnet-base-nli-mean-tokens',
    'electra-base-nli-mean-tokens'
]

results = {}
for model_name in models:
    accuracy, f1, inference_time = evaluate_model(model_name, train_texts, train_labels, test_texts, test_labels)
    results[model_name] = {
        'accuracy': accuracy,
        'f1': f1,
        'inference_time': inference_time
    }


Resource utilization

In [ ]:
import torch

def measure_resource_utilization(model_name):
    # Load the model
    model = SentenceTransformer(model_name)

    # Move model to GPU if available
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    model = model.to(device)

    # Measure GPU memory usage before and after encoding
    if device == 'cuda':
        torch.cuda.reset_peak_memory_stats()
        _ = model.encode(train_texts[:10], convert_to_tensor=True)
        peak_memory = torch.cuda.max_memory_allocated() / (1024 ** 2)  # Convert to MB
    else:
        peak_memory = 0  # No GPU available

    return peak_memory

for model_name in models:
    peak_memory = measure_resource_utilization(model_name)
    print(f"Model: {model_name}, Peak Memory Usage: {peak_memory:.2f} MB")
